In [1]:
import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10

In [2]:
# Load CIFAR-10 data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [3]:
print(f"Training Features Shape: {X_train.shape}") # Expected: (50000, 32, 32, 3)
print(f"Training Labels Shape: {y_train.shape}")   # Expected: (50000, 1)

Training Features Shape: (50000, 32, 32, 3)
Training Labels Shape: (50000, 1)


In [4]:
# Define the 10 class names for later interpretation
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [5]:
# 1. Normalize the image data
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [6]:
# 2. One-Hot Encode the labels
num_classes = 10
y_train_encoded = to_categorical(y_train, num_classes)
y_test_encoded = to_categorical(y_test, num_classes)

In [7]:
print(f"Encoded Labels Shape: {y_train_encoded.shape}") # Expected: (50000, 10)

Encoded Labels Shape: (50000, 10)


In [8]:
from tensorflow.keras.models import Sequential
model = Sequential()

In [9]:
# Layer 1: Convolution and Pooling
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Layer 2: Deeper Convolution and Pooling
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [11]:
# Layer 3: Final Convolutional Block
model.add(Conv2D(128, (3, 3), activation='relu'))

In [12]:
# Transition to ANN (Fully Connected)
model.add(Flatten())
model.add(Dense(512, activation='relu'))

In [13]:
# Dropout Layer (Crucial for complex models to prevent overfitting)
model.add(Dropout(0.5))

In [14]:
# Output Layer
model.add(Dense(num_classes, activation='softmax'))

In [15]:
# Print the model structure
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,147,466 (4.38 MB)

 Trainable params: 1,147,466 (4.38 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
from tensorflow.keras.optimizers import Adam

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy', # Standard loss for multi-class classification
              metrics=['accuracy'])

# Train the model
# NOTE: Set epochs higher (e.g., 20 or more) for real training. Using 5 for a quick demo.
history = model.fit(X_train, y_train_encoded,
                    epochs=5,
                    batch_size=64,
                    validation_split=0.1, # Use 10% of training data for validation during training
                    verbose=1)

print("\nModel training complete.")

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 83s 115ms/step - accuracy: 0.3332 - loss: 1.7952 - val_accuracy: 0.5540 - val_loss: 1.2492
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 80s 113ms/step - accuracy: 0.5552 - loss: 1.2511 - val_accuracy: 0.6036 - val_loss: 1.1040
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 82s 116ms/step - accuracy: 0.6259 - loss: 1.0619 - val_accuracy: 0.6102 - val_loss: 1.1179
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 84s 118ms/step - accuracy: 0.6670 - loss: 0.9548 - val_accuracy: 0.6756 - val_loss: 0.9062
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 136s 110ms/step - accuracy: 0.7037 - loss: 0.8420 - val_accuracy: 0.6902 - val_loss: 0.8787

Model training complete.


In [17]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test_encoded, verbose=0)

print(f"\n--- Final Test Evaluation ---")
print(f"Test Loss: {loss:.4f}")
# Typical Accuracy for this simple CNN on CIFAR-10 is around 65% - 75%
print(f"Test Accuracy: {accuracy*100:.2f}%")

# Make a sample prediction (e.g., for the first test image)
predictions = model.predict(X_test[:1])
predicted_class_index = np.argmax(predictions[0])
true_class_index = y_test[0][0]

print("\n--- Sample Prediction ---")
print(f"Model Predicted Index: {predicted_class_index} ({class_names[predicted_class_index]})")
print(f"True Index: {true_class_index} ({class_names[true_class_index]})")


--- Final Test Evaluation ---
Test Loss: 0.8898
Test Accuracy: 68.77%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step

--- Sample Prediction ---
Model Predicted Index: 3 (cat)
True Index: 3 (cat)
